In [75]:
import numpy as np
import pyphi

from qutip import *
from qutip_qip.operations import *
from intrinsic_difference import intrinsic_difference
from utils import *
from compute_ces import *

In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Define states and matrices

In [2]:
# Basis states (for both classical and quantum)
zero = basis(2,0)
one = basis(2,1)

# Normalized fully general quantum state
plus = (zero + one).unit()
minus = (zero - one).unit()

# Density matrix representations of the states
rho_zero = zero * zero.dag() # could also be fock_dm(2,0)
rho_one = one * one.dag()
rho_plus = plus * plus.dag()
rho_minus = minus * minus.dag()
rho_mm = 0.5 * rho_zero + 0.5 * rho_one

In [3]:
print('ID |0> : ', intrinsic_difference(rho_zero, rho_mm))
print('ID |+> : ', intrinsic_difference(rho_plus, rho_mm))

ID |0> :  (1.0, [array([-1.+0.j, -0.+0.j])])
ID |+> :  (1.0, [array([0.70710678+0.j, 0.70710678+0.j])])


# 2 Qubits - CNOT

In [4]:
cnot()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

## Classical results

In [77]:
pyphi.config.MEASURE = 'ID'
pyphi.config.PARTITION_TYPE = 'ALL'
pyphi.config.PICK_SMALLEST_PURVIEW = False

In [78]:
print(pyphi.config)

{ 'ACTUAL_CAUSATION_MEASURE': 'PMI',
  'ASSUME_CUTS_CANNOT_CREATE_NEW_CONCEPTS': False,
  'CACHE_POTENTIAL_PURVIEWS': True,
  'CACHE_REPERTOIRES': True,
  'CACHE_SIAS': False,
  'CACHING_BACKEND': 'fs',
  'CLEAR_SUBSYSTEM_CACHES_AFTER_COMPUTING_SIA': False,
  'CUT_ONE_APPROXIMATION': False,
  'FS_CACHE_DIRECTORY': '__pyphi_cache__',
  'FS_CACHE_VERBOSITY': 0,
  'LOG_FILE': 'pyphi.log',
  'LOG_FILE_LEVEL': 'INFO',
  'LOG_STDOUT_LEVEL': 'WARNING',
  'MAXIMUM_CACHE_MEMORY_PERCENTAGE': 50,
  'MEASURE': 'ID',
  'MONGODB_CONFIG': { 'collection_name': 'cache',
                      'database_name': 'pyphi',
                      'host': 'localhost',
                      'port': 27017},
  'NUMBER_OF_CORES': -1,
  'PARALLEL_COMPLEX_EVALUATION': False,
  'PARALLEL_CONCEPT_EVALUATION': False,
  'PARALLEL_CUT_EVALUATION': True,
  'PARTITION_TYPE': 'ALL',
  'PICK_SMALLEST_PURVIEW': False,
  'PRECISION': 6,
  'PRINT_FRACTIONS': True,
  'PROGRESS_BARS': True,
  'REDIS_CACHE': False,
  'REDIS_CONFIG'

In [7]:
CNOT_tpm = cnot().data.toarray()
#CNOT_tpm = pyphi.convert.le2be_state_by_state(CNOT_tpm.real)

In [8]:
CNOT_tpm.real

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [9]:
CNOT_tpm = pyphi.convert.be2le_state_by_state(CNOT_tpm.real)

In [10]:
CNOT_tpm = pyphi.convert.to_2dimensional(pyphi.convert.state_by_state2state_by_node(CNOT_tpm.real))

In [11]:
network = pyphi.Network(CNOT_tpm)
subsystem = pyphi.Subsystem(network, (1,0))

In [12]:
ces = pyphi.compute.ces(subsystem)

In [13]:
ces

════════════════════════════════════════════════════════════
            Cause-effect structure (2 concepts)             
════════════════════════════════════════════════════════════
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              Concept: Mechanism = [n0], φ = 1              
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              MIC                         MIE               
  ┌──────────────────────────┐┌──────────────────────────┐  
  │  φ = 1                   ││  φ = 1                   │  
  │  Purview = [n0]          ││  Purview = [n0]          │  
  │  MIP:                    ││  MIP:                    │  
  │    n0     ∅              ││    n0     ∅              │  
  │    ─── ✕ ───             ││    ─── ✕ ───             │  
  │     ∅    n0              ││     ∅    n0              │  
  │  Repertoire:             ││  Repertoire:             │  
  │    ┌─────────────┐       ││    ┌─────────────┐       │  
  │    │ S    Pr(S)  │  

## Quantum results

In [79]:
rho = tensor(rho_one, rho_zero)

In [80]:
rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]

In [81]:
len(rho.dims[0])

2

In [82]:
rho_e10 = evolve(tensor(rho_one, rho_zero), cnot(), direction = 'effect')

In [83]:
rho_e10

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]

In [84]:
partitions = pyphi.partition.mip_partitions((0,1), (0,1))

In [39]:
partitions[1][0].mechanism == (0,)

True

In [35]:
tensor(rho.ptrace(list((1,))), rho_mm)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0. ]
 [0.  0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]

In [28]:
partitions = [p for p in partitions]

In [20]:
[print(p) for p in pyphi.utils.powerset(range(2), nonempty=True)]

(0,)
(1,)
(0, 1)


[None, None, None]

In [99]:
result = compute_ces(rho, cnot())

m:  (0,)


TypeError: 'int' object is not iterable

In [96]:
result

[{'mech': (0,),
  'purview': (0,),
  'phi': 1.0,
  'mip':  0     ∅ 
  ─── ✕ ───
   ∅     0 ,
  'state': [array([0.+0.j, 1.+0.j])]},
 {'mech': (1,),
  'purview': (0, 1),
  'phi': 0.5,
  'mip':  1     ∅ 
  ─── ✕ ───
   ∅    0,1,
  'state': [array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
   array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])]},
 {'mech': (0, 1),
  'purview': (0, 1),
  'phi': 1.0,
  'mip':  0     1     ∅ 
  ─── ✕ ─── ✕ ───
  0,1    ∅     ∅ ,
  'state': [array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])]}]

In [50]:
parts

[(Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
  Qobj data =
  [[0.  0.  0.  0. ]
   [0.  0.  0.  0. ]
   [0.  0.  0.5 0. ]
   [0.  0.  0.  0.5]],
  (0, 1)),
 (Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
  Qobj data =
  [[0.5 0.  0.  0. ]
   [0.  0.  0.  0. ]
   [0.  0.  0.  0. ]
   [0.  0.  0.  0.5]],
  (0, 1)),
 (Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
  Qobj data =
  [[0. 0.]
   [0. 1.]],
  (0,)),
 (Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
  Qobj data =
  [[0.5 0. ]
   [0.  0.5]],
  (1,)),
 (Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
  Qobj data =
  [[0.5 0. ]
   [0.  0.5]],
  (1,)),
 (Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
  Qobj data =
  [[0.5 0. ]
   [0.  0.5]],
  (1,)),
 (Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
  Qobj

In [52]:
tensor([rho_mm for i in range(4)])

Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[0.0625 0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.    ]
 [0.     0.0625 0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.0625 0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.0625 0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.0625 0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.0625 0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.0625 0.     0.     0.
  0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.0625 0.     0.
  0.     0.     0.     0.     0

In [ ]:
parts[2]

In [ ]:
toffoli()

In [ ]:
A = [p for p in pyphi.partition.mip_partitions([0,1], [2,3])]

In [ ]:
A[0][0].purview

In [ ]:
A[2]

In [ ]:
for part in A[2]:
    if len(part.mechanism) < 1:
        # purview is max ent
        rho_p = tensor([rho_mm for p in part.purview])

In [ ]:
tensor(rho_one, rho_p).data.toarray().real.reshape(2,2,2)

In [ ]:
Q = Qobj(rho_p.data.toarray(), dims = [[2,1,2], [2,1,2]])

In [ ]:
rho_p.data.toarray().reshape(())

In [ ]:
len(A[0][0].mechanism)

In [ ]:
Qobj([[0.5, 0.],[0., 0.5]])

In [ ]:
np.random.rand(2,2)